<a href="https://colab.research.google.com/github/merucode/DL/blob/81-colab-keggle_image/01-01_%5Bimage-classification-CNN%5D_Aerial-Cactus_Identification(basic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imformation

* Title : [Aerial Cactus Identification](https://www.kaggle.com/c/aerial-cactus-identification)
* Type : image binary classification
* Evaluation : ROE ACU
* Model : CNN
* Python version: 3.10.6
* Library version
  * torch(torch==2.0.1+cu118)
  * torchvision(torchvision==0.15.2+cu118)
  * sklearn(scikit-learn==1.2.2)
  * cv2(opencv-python==4.7.0.72)
  * numpy(numpy==1.22.4)
  * pandas(pandas==1.5.3)
  * matplotlib(matplotlib==3.7.1)
  * zipfile, random, math, shutil, os

# STEP 0. Version check and Install Dependency

Step 0-1. Install Dependency

In [ ]:
!pip install opencv--python

Step 0-2. Version Check

In [ ]:
import sys
import torch
print(f"Python version:{sys.versioㅐn}")                  # python
print("Torch version:{}".format(torch.__version__))     # torch
print("cuda version: {}".format(torch.version.cuda))    # cuda
print("cudnn version:{}".format(torch.backends.cudnn.version()))    # cudnn

In [ ]:
!pip list

Step 0-3. Install Data

In [ ]:
!export KAGGLE_USERNAME=merucode && export KAGGLE_KEY=929873bded61bcd9be3a12e4ec85e04b && kaggle competitions download -c aerial-cactus-identification

In [ ]:
!unzip aerial-cactus-identification.zip

In [ ]:
from zipfile import ZipFile

data_path = '/content/'

with ZipFile(data_path + 'train.zip') as zipper:
  zipper.extractall()

with ZipFile(data_path + 'test.zip') as zipper:
  zipper.extractall()

# STEP 1. Check Data

Step 1-1. Check data

In [ ]:
import pandas as pd

data_path = '/content/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [ ]:
labels.head(3)

In [ ]:
submission.head(3)

Step 1-2. Data Visualize

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

mpl.rc('font', size=15)
plt.figure(figsize=(7, 7))
label = ['Has cactus', 'Hasn\'t cactus']  # 타깃값 레이블
# 타깃값 분포 파이 그래프
plt.pie(labels['has_cactus'].value_counts(), labels=label, autopct='%.1f%%')

In [ ]:
import os

num_train = len(os.listdir('train/'))
num_test = len(os.listdir('test/'))

print(f'훈련 데이트 갯수: {num_train}')
print(f'테스트 데이트 갯수: {num_test}')

Step 1-3. Data Image Visualize

In [ ]:
import matplotlib.gridspec as gridspec
import cv2 # OpenCV 라이브러리

# 선인장을 포함하는 이미지 파일명(마지막 12개)
last_has_cactus_img_name = labels[labels['has_cactus']==1]['id'][-12:]

def image_show(path, img_names):
  mpl.rc('font', size=7)
  plt.figure(figsize=(15, 6)) # 전체 Figure 크기 설정
  grid = gridspec.GridSpec(2, 6) # 서브플롯 배치

  for idx, img_name in enumerate(img_names):
    img_path = path + img_name                      # 이미지 파일 경로
    image = cv2.imread(img_path)                    # 이미지 파일 읽기
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # 이미지 색상 보정
    ax = plt.subplot(grid[idx])
    ax.imshow(image)                                # 이미지 출력

# 이미지 출력
image_show('train/', last_has_cactus_img_name)

In [ ]:
# 선인장을 포함하지 않는 이미지 파일명(마지막 12개)
last_hasnt_cactus_img_name = labels[labels['has_cactus']==0]['id'][-12:]

# 이미지 출력
image_show('train/', last_hasnt_cactus_img_name)

# STEP 2. Setting for Modeling

Step 2-1. Seed

In [ ]:
import torch
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)                 # 파이썬 난수 생석이 시드 고정
np.random.seed(seed)              # 넘파이 난수 생성기 시드 고정
torch.manual_seed(seed)           # 파이토치 난수 생성기 시드 고정(CPU 사용시)
torch.cuda.manual_seed(seed)      # 파이토치 난수 생성기 시드 고정(GPU 사용시)
torch.cuda.manual_seed_all(seed)  # 파이토치 난수 생성기 시드 고정(멀티 GPU 사용 시)
torch.backends.cudnn.deterministic = True # 확정적 연산 사용
torch.backends.cudnn.benchmark = False    # 벤치마크 기능 해제
torch.backends.cudnn.enabled = False      # cudnn 사용 해제

Step 2-2.GPU 장비 설정

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

# STEP 3. Dataset

Step 3-1. Load Data

In [ ]:
import pandas as pd

data_path = '/content/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [ ]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 세트 분리
train, valid = train_test_split(labels,
                                test_size=0.1,                 # 9:1 비율로 test 세트 생성
                                stratify=labels['has_cactus'], # 훈련 데이터, 검증 데이터 티깃값 비율 3:1 유지
                                random_state=50)

print(f"훈련 데이터 개수: {len(train)}")
print(f"검증 데이터 개수: {len(valid)}")
train.head(3)

Step 3-2. Dataset

In [ ]:
import cv2
from torch.utils.data import Dataset

class ImageDataset(Dataset):
  # 초기화 메서드(생성자)
  def __init__(self, df, img_dir='./', transform=None):
    super().__init__() # 상송받은 Dataset 생성자 호출
    # 전달받은 인수들 저장
    self.df = df
    self.img_dir = img_dir
    self.transform = transform

  # 데이터셋 크기 반환 메서드
  def __len__(self):
    return len(self.df)

  # idx 해당하는 데이터 반환 메서드
  def __getitem__(self, idx):
    img_id = self.df.iloc[idx, 0]       # 이미지 ID
    img_path = self.img_dir + img_id    # 이미지 파일 경로
    image = cv2.imread(img_path)        # 이미지 파일 읽기
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # 이미지 색상 보정
    label = self.df.iloc[idx, 1]        # 이미지 레이블(타깃값)

    if self.transform is not None:
      image = self.transform(image)     # 변화기가 있다면 이미지 변환

    return image, label

In [ ]:
from torchvision import transforms # 이미지 변환을 위한 모듈

transform = transforms.ToTensor()

dataset_train = ImageDataset(df=train, img_dir='train/', transform=transform)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform)

Step 3-3. Dataloader

In [ ]:
from torch.utils.data import DataLoader # 데이터 로더 클래스

loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_vaild = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

# STEP 4. Module

In [ ]:
import torch.nn as nn             # 신경망 모듈
import torch.nn.functional as F   # 신경망 모듈에서 자주 사용되는 함수

class Model(nn.Module):
  # 신경망 계층 정의
  def __init__(self):
    super().__init__() # 상속받은 nn.Module의 __init__() 메서드 호출

    # 첫 번째 합성곱 계층
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32,
                           kernel_size=3, padding=2)

    # 두 번째 합성곱 계층
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64,
                           kernel_size=3, padding=2)

    # 최대 풀링 계층()
    self.max_pool = nn.MaxPool2d(kernel_size=2)

    # 평균 풀링 계층
    self.avg_pool = nn.AvgPool2d(kernel_size=2)

    # 전결합 계층
    self.fc = nn.Linear(in_features=64*4*4, out_features=2)

  # 순전파 출력 정의
  def forward(self, x):
    x = self.max_pool(F.relu(self.conv1(x)))  # 3*32*32  > 32*34*34 > 32*17*17
    x = self.max_pool(F.relu(self.conv2(x)))  # 32*17*17 > 64*19*19 > 64*9*9
    x = self.avg_pool(x)                      # 64*9*9 > 64*4*4
    x = x.view(-1, 64*4*4)  # 평탄화          # 64*4*4 > 1024
    x = self.fc(x)                            # 1024 > 2
    return x

# STEP 5. Learning

Step 5-1. Setting

In [ ]:
import tqdm
from torch.optim.adam import Adam

device = "cuda" if torch.cuda.is_available() else "cpu"
model = Model().to(device)

# 손실 함수
criterion = nn.CrossEntropyLoss()
# 옵티마이저
optim = Adam(model.parameters(), lr=0.01)

Step 5-2. Learning

In [ ]:
epochs = 10 # 총 에폭

for epoch in range(epochs):
  epoch_loss = 0 # 에폭별 손실값 초기화

  # 반복 횟수 만큼 반복
  for images, labels in loader_train:
    # 이미지, 레이블 데이터 미니배치를 장비에 할당
    images = images.to(device)
    labels = labels.to(device)

    optim.zero_grad()         # 옵티마이저 기울기 초기화
    outputs = model(images)   # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
    loss = criterion(outputs, labels)   # 손실값 계산(예측값과 타깃값의 오차)
    epoch_loss += loss.item() # 현재 배치에서의 손실 추가
    loss.backward()           # 역전파 수행
    optim.step()     # 가중치 갱신

  # 훈련 데이터 손실값 출력
  print(f"에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss/len(loader_train):.4f}")

# STEP 6. Validation

Step 6-1. Setting

In [ ]:
from sklearn.metrics import roc_auc_score # ROC ACU
# 실제값과 예측 확률값을 담을 리스트 초기화
true_list = []
preds_list = []

# 모델을 평가 상태로 설정
model.eval()

Step 6-2. Model Validation

In [ ]:
with torch.no_grad(): # 기울기 계산 비활성화
  for images, labels in loader_vaild:
    # 이미지, 레이블 데이터 미니배치를 장비에 활당
    images = images.to(device)
    labels = labels.to(device)

    # 순전파
    outputs = model(images) # output 2개로 구성(타깃값 0에 대한 출력값, 타깃값 1에 대한 출력값)
    preds = torch.softmax(outputs.cpu(), dim=1)[:, 1] # 예측값 1에 대한 예측 확률
    true = labels.cpu() # 실제값

    # 예측 확률과 실제값을 리스트에 추가
    preds_list.extend(preds)
    true_list.extend(true)


# 검증 데이터 ROC AUC 점수 계산
print(f"검증 데이터 ROU AUC: {roc_auc_score(true_list, preds_list):.4f}")

# STEP 7. Evaluation and Submission

Step 7-1. Setting

In [ ]:
dataset_test = ImageDataset(df=submission, img_dir='test/', transform=transform)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=False)

Step 7-2. Evaluation

In [ ]:
model.eval()  # 모델 평가 상태로 설정

preds = []    # 타깃 예측값 저장용 리스트 초기화

with torch.no_grad(): # 기울기 계산 비활성화
  for images, _ in loader_test:
    # 이미지 데이터 미니배치를 장비에 할당
    images = images.to(device)

    # 순전파
    outputs = model(images)
    # 타깃값이 1일 확률(예측값)
    preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
    # preds에 preds_part 이어 붙이기
    preds.extend(preds_part)

Step 7-3. Submission

In [ ]:
submission['has_cactus'] = preds                  # submission df 결과값 재설정
submission.to_csv('submission.csv', index=False)  # 제출 파일 생성

# # 이미지 테스트 데이터 삭제
# import shutil

# shutil.rmtree('./train')
# shutil.rmtree('./test')

In [ ]:
submission.head(3)